In [ ]:
pip install openai langchain chromadb pypdf sentence-transformers

In [2]:
# Step 0 - fetch OpenAI API key from the environment
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
# Step 1 - load documents
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://www.lords.org/getattachment/MCC/All-Laws/2nd-Edition-of-the-2017-code-2019.pdf")
documents = loader.load()

In [11]:
# Step 2 - split documnets into chunks
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [12]:
# Step 3 - load documents into Chroma using the open-source embedding function
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(docs, embedding_function)

In [13]:
# Step 4 - define the LLM model
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"


from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, 
                 temperature=0)

In [14]:
# Step 5 - create the chain with the LLM model and the database
from langchain.chains import RetrievalQA

retrievalQA = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=db.as_retriever(), 
    verbose=True
)

In [16]:
# Step 6 - ask the question and display the response!
from IPython.display import display, Markdown

prompt = "According to given context, create a list of the different ways a batsman can get out"
response = retrievalQA.run(prompt)

display(Markdown(response))



> Entering new RetrievalQA chain...

> Finished chain.


Based on the given context, here is a list of different ways a batsman can get out in cricket:

1. Bowled
2. Caught
3. Stumped
4. Hit the ball twice
5. LBW (Leg Before Wicket)
6. Hit wicket
7. Run out
8. Obstructing the field

Please note that this list may not be exhaustive and there may be other ways a batsman can get out in cricket.